

![title](images/lostlake_gorgeflyshop.jpg)
Lost Lake, Mt Hood National Forest. Image credit: Gorge Fly Shop  

### Lets start with some requirements before we code
* Want to know where we can go camping, lets start with near Mount Hood
* Tabular structure will enable fast querying, start out with a dataframe

What campground features do we want to capture?

* Toilet facilities
* Near water
* Accessible
* Pets allowed
* Campground status - open, closed
* Located


In [77]:
from csv import DictReader
import geopandas as gpd
import json
import numpy as np
import pandas as pd
import itertools

from camping.mocks.request import RequestsMock
from camping.util.scraper import Scraper
from camping.util.distance import distance_merge

def max_col_width(w=100):
    pd.set_option('display.max_colwidth', w)

ridb_facilities_url = "https://ridb.recreation.gov/api/v1/facilities"

#### RIDB - Recreation Information Database - Recreation.gov

[RIDB](https://ridb.recreation.gov/) is a data service spanning multiple US government agencies surfacing data related to recreational opportunities across the country.

##### Facilities endpoint: `https://ridb.recreation.gov/api/v1/facilities`
Get a list of facilities by activity. Camping is `activity_id=9`  
Query for facilities nearby a specific point via lat, long, and radius  
Query for facilities in a given state by supplying the 2 digit state code  

##### Campsites endpoint: `https://ridb.recreation.gov/api/v1/facilities/{facility_id}/campsites`
Campsite specific information for a given facility. Returns a list of campsites at the facility

https://ridb.recreation.gov/docs  

Make sure you are making appropriate use of resources: 
https://ridb.recreation.gov/ridb-access-agreement 

RIDB has an API, which you can access with a Recreation.gov account. In this lab we will use a mock of the API, so you don't need an API key.

If you want to work with live data, go to the Profile area under your account and click the Generate New API key account. Place your key in the `apiKey` field of the `headers` dict below and replace `RequestsMock` with `requests` and you should be good to go!

Lets take a look at RIDB facilities with camping near Mount Hood Oregon

In [32]:
# NOTE: If using the mock not change params, mock looks for lat/long/radius
params = {"activity_id":9, "latitude":45.4977712, "longitude":-121.8211673, "radius":15}
headers = {"accept": "application/json", "apikey": "key"}
response = RequestsMock.get(ridb_facilities_url, params, headers=headers)
camping_json  = json.loads(response.text)
df_ridb_camping = pd.DataFrame(camping_json['RECDATA'])
df_ridb_camping.head()

,FacilityID,LegacyFacilityID,OrgFacilityID,ParentOrgID,ParentRecAreaID,FacilityName,FacilityDescription,FacilityTypeDescription,FacilityUseFeeDescription,FacilityDirections,...,FacilityMapURL,FacilityAdaAccess,GEOJSON,FacilityLongitude,FacilityLatitude,Keywords,StayLimit,Reservable,Enabled,LastUpdatedDate
0,234306,75167,AN375167,131,1102,EAGLE CREEK OVERLOOK GRP SITE,<h2>Overview</h2>\nEagle Creek Overlook Group Site is set on a forested bluff above the Columbia...,Campground,,"From Portland, OR; Travel east on Interstate 84. Take exit #41 ""Eagle Creek Recreation Area"", ju...",...,,N,"{'TYPE': 'Point', 'COORDINATES': [-121.9308333, 45.6416667]}",-121.930833,45.641667,"ECOG,Overlook,Eagle Creek Overlook,Overlook Shelter,Eagle Creek Overlook Shelter,COLUMBIA R GORG...",,True,True,2021-04-13
1,232834,71617,AN371617,131,1106,RILEY HORSE CAMPGROUND,<h2>Overview</h2>\nRiley Campground is an equestrian facility located northwest of Oregon's maje...,Campground,,"From Sandy, travel Highway 26 for 18 miles to Zigzag. Turn north onto Lolo Pass Road. Continue f...",...,,N,"{'TYPE': 'Point', 'COORDINATES': [-121.8594444, 45.3813889]}",-121.859444,45.381389,"RILE,MT. HOOD NF - FS",,True,True,2021-04-13
2,234075,74082,AN374082,126,16835,WILDWOOD RECREATION SITE,"<h2>Overview</h2>\n<p>Motorized access to <strong>Wildwood is OPEN</strong>, however, temporary ...",Campground,<ul>\n<li>A daily vehicle fee is not included with shelter reservations. Fees vary from $5 (unde...,"Wildwood Recreation Site is located 39 miles east of Portland on Highway 26 near Welches, Oregon.\n",...,,N,"{'TYPE': 'Point', 'COORDINATES': [-121.9866667, 45.3561111]}",-121.986667,45.356111,"WIWO,Wildwood Recreation Site,Wild Wood Recreation Site,Wildwood Rec Site,Wild Wood Rec Site,Wil...",,True,True,2021-04-13
3,233329,72357,AN372357,131,1106,TILLY JANE GUARD STATION,<h2>Overview</h2>\nTilly Jane Guard Station is an ideal getaway for individuals and families who...,Campground,"<p>Reservation Fee: $6.00 (non-refundable)</p>\n<p>Recreation Fee: $200.00 per night, two night...","From Hood River, Oregon, travel south on Highway 35 for 8.5 miles, past the Hood River Ranger St...",...,,N,"{'TYPE': 'Point', 'COORDINATES': [-121.6480556, 45.3988889]}",-121.648056,45.398889,"Guard Station,Cloud Cap Guard Station",,True,True,2021-04-13
4,122940,122940,,131,1102,WYETH CAMPGROUND,<h2>Overview</h2>\nWyeth Campground is just off of Interstate 84 in a forested setting with gras...,Campground,,"From east or west on Oregon Interstate 84, take Wyeth exit #51. Turn west on Herman Creek Road....",...,,N,"{'TYPE': 'Point', 'COORDINATES': [-121.7722222, 45.6902778]}",-121.772222,45.690278,,,True,True,2021-04-13


We have an idea of what we can get from the facilities endpoint:
* Facility Name & Facility ID
* Lat/Long
* Ada Accessability
* Description

Lets get more specific data on campsites at once of these facilities
Insert lost lake pic

In [9]:
df_ridb_camping.query("FacilityName == 'LOST LAKE RESORT AND CAMPGROUND'")['FacilityID']

7    251434
Name: FacilityID, dtype: object

In [123]:
resp = RequestsMock.get(f"{ridb_facilities_url}/251434/campsites", headers=headers)
campsites_json = json.loads(resp.text)
df_campsites = pd.DataFrame(campsites_json['RECDATA'])
df_campsites.head()

,CampsiteID,FacilityID,CampsiteName,CampsiteType,TypeOfUse,Loop,CampsiteAccessible,CampsiteLongitude,CampsiteLatitude,CreatedDate,LastUpdatedDate,ATTRIBUTES,ENTITYMEDIA,PERMITTEDEQUIPMENT
0,96115,251434,A021,RV NONELECTRIC,Overnight,A Loop,False,-121.816561,45.498111,2015-04-09,2020-10-15,"[{'AttributeName': 'Driveway Grade', 'AttributeValue': 'Slight'}, {'AttributeName': 'Max Num of ...","[{'EntityMediaID': '567f9beb-d16e-4317-b7e1-28916afbe40d', 'MediaType': 'Image', 'EntityID': '96...","[{'EquipmentName': 'Trailer', 'MaxLength': 81}, {'EquipmentName': 'Tent', 'MaxLength': 81}, {'Eq..."
1,96043,251434,C005,RV NONELECTRIC,Overnight,C Loop,True,-121.813871,45.496703,2015-04-09,2020-10-15,"[{'AttributeName': 'Driveway Entry', 'AttributeValue': 'Pull-Through'}, {'AttributeName': 'Place...","[{'EntityMediaID': '9d9d71ba-a3c2-41b8-b11c-b65b8054860a', 'MediaType': 'Image', 'EntityID': '96...","[{'EquipmentName': 'Trailer', 'MaxLength': 54}, {'EquipmentName': 'Tent', 'MaxLength': 54}, {'Eq..."
2,96075,251434,D001,RV NONELECTRIC,Overnight,D Loop,True,-121.815180,45.494484,2015-04-09,2020-10-15,"[{'AttributeName': 'Accessibility', 'AttributeValue': 'Y'}, {'AttributeName': 'Site Access', 'At...","[{'EntityMediaID': '7c833b0a-b0cd-4093-8c91-95a1b9e12311', 'MediaType': 'Image', 'EntityID': '96...","[{'EquipmentName': 'Trailer', 'MaxLength': 36}, {'EquipmentName': 'Tent', 'MaxLength': 36}, {'Eq..."
3,96104,251434,A009,RV NONELECTRIC,Overnight,A Loop,False,-121.816130,45.498334,2015-04-09,2020-10-15,"[{'AttributeName': 'Base Number of People', 'AttributeValue': '0'}, {'AttributeName': 'Shade', '...","[{'EntityMediaID': 'd4cec022-4663-4245-a823-a09925f1dcef', 'MediaType': 'Image', 'EntityID': '96...","[{'EquipmentName': 'Trailer', 'MaxLength': 34}, {'EquipmentName': 'Tent', 'MaxLength': 34}, {'Eq..."
4,96032,251434,H001,EQUESTRIAN NONELECTRIC,Overnight,Horsecamp,True,-121.811069,45.484083,2015-04-09,2020-10-15,"[{'AttributeName': 'Horse Stall/Corral', 'AttributeValue': 'Y'}, {'AttributeName': 'Horse Hitchi...","[{'EntityMediaID': '2590bc58-4289-4a98-934a-6f91d1224ba0', 'MediaType': 'Image', 'EntityID': '96...","[{'EquipmentName': 'Horse', 'MaxLength': 32}]"


Campsite ATTRIBUTES - JSON describing different characteristics of each campsite  
What characteristics are available?

In [128]:
set(itertools.chain(*df_campsites['ATTRIBUTES'].apply(lambda x: [entry['AttributeName'] for entry in x])))

{'Accessibility',
 'BBQ',
 'Base Number of People',
 'Base Number of Vehicles',
 'Campfire Allowed',
 'Capacity/Size Rating',
 'Checkin Time',
 'Checkout Time',
 'Double Driveway',
 'Driveway Entry',
 'Driveway Grade',
 'Driveway Length',
 'Driveway Surface',
 'Fire Pit',
 'Grills/Fire Ring',
 'Hike In Distance to Site',
 'Horse Hitching Post',
 'Horse Stall/Corral',
 'IS EQUIPMENT MANDATORY',
 'Location Rating',
 'Max Num of People',
 'Max Num of Vehicles',
 'Max Vehicle Length',
 'Min Num of People',
 'Pets Allowed',
 'Picnic Table',
 'Placed on Map',
 'Platform',
 'Privacy',
 'Proximity to Water',
 'Quiet Area',
 'Shade',
 'Site Access',
 'Site Height/Overhead Clearance',
 'Site Rating',
 'Tent Pad',
 'Tent Pad Length',
 'Tent Pad Width'}

#### Transforming campsite data

To make it easier to search for campsites by attribute, we will convert the ATTRIBUTES to columns  
Also, limit the columns we retain to only those we need

In [175]:
def transform_campsites(campsite_json):
    # First, translate the ATTRIBUTES list into a list of dict of {AttributeName: AttributeValue}
    for i in range(len(campsite_json)):
        campsite_json[i]['AttributeDict'] = [{item['AttributeName']: item['AttributeValue']} for item in campsite_json[i]['ATTRIBUTES']]

    # Next, convert the AttriuteDict to columns
    df = pd.DataFrame(campsites['RECDATA'])[['ATTRIBUTES', 'CampsiteID', 'CampsiteName', 'FacilityID']]
    df['AttributeDict'] = df['ATTRIBUTES'].apply(lambda x: {item['AttributeName']: item['AttributeValue'] for item in x})
    norm = pd.json_normalize(df['AttributeDict'])
    df = df[['CampsiteID','CampsiteName','FacilityID']].join(norm)
    return df.replace({np.nan: ""})

In [176]:
d = transform_campsites(campsites['RECDATA'])

In [177]:
d.query("`Proximity to Water` != '' & `Fire Pit` != ''")

,CampsiteID,CampsiteName,FacilityID,Driveway Grade,Max Num of Vehicles,Fire Pit,Privacy,Base Number of Vehicles,Base Number of People,Grills/Fire Ring,...,Hike In Distance to Site,Site Access,Accessibility,Double Driveway,Horse Stall/Corral,Horse Hitching Post,BBQ,Platform,Proximity to Water,Min Num of People
15,96099,F004,251434,Slight,1,Y,Y,0,0,Y,...,275,Hike-In,Y,,,,Y,Y,Lakefront,
19,96278,F005,251434,Slight,1,Y,Y,0,0,Y,...,300,Hike-In,Y,,,,Y,Y,Lakefront,
23,96224,F006,251434,Slight,1,Y,Y,0,0,Y,...,325,Hike-In,Y,,,,Y,Y,Lakefront,
41,96264,F002,251434,Slight,1,Y,Y,0,0,Y,...,225,Hike-In,Y,,,,Y,Y,Lakefront,
45,96303,F001,251434,Slight,1,Y,Y,0,0,Y,...,200,Hike-In,Y,,,,Y,Y,Lakefront,


In [21]:
# putting it all together
ridb_facilities_url = "https://ridb.recreation.gov/api/v1/facilities"
params = {"activity_id":9, "state":"OR"}
headers = {"accept": "application/json", "apikey": "key"}


response = RequestsMock.get(ridb_facilities_url, params, headers=headers)
camping_json  = json.loads(response.text)

# Do we really need the campgrounds in a dataframe?
df_ridb_camping = pd.DataFrame(camping_json['RECDATA'])

campground_info = pd.DataFrame()
for facility in camping_json['RECDATA']:
    if facility.get('FacilityID') is not None:
        campground_url = f"{ridb_facilities_url}/{facility['FacilityID']}/campsites"
        resp = RequestsMock.get(campground_url, headers=headers)
        if resp.status_code != 200:
            continue
        
        campsites = json.loads(resp.text)
        if len(campsites['RECDATA']) > 0:
            df_campsites = pd.DataFrame(campsites['RECDATA'])
            campground_info = campground_info.append(df_campsites[['FacilityID', 'CampsiteID', 'CampsiteName', 'ATTRIBUTES']].merge(df_ridb_camping, on='FacilityID', how='left'))

In [22]:
campground_info['FacilityName'].unique()

array(['EAST LEMOLO CAMPGROUND', 'MAGONE LAKE', 'EAST DAVIS CAMPGROUND',
       'CLEATOR BEND', 'ASPEN POINT', 'DRAKE PEAK LOOKOUT',
       'Anthony Lake', 'THREE CREEKS MEADOW HORSE CAMP',
       'NORTH FORK CAMPGROUND (OR)', 'TILLY JANE A-FRAME',
       'MUSICK GUARD STATION', 'LOST LAKE RESORT AND CAMPGROUND'],
      dtype=object)

In [ ]:
campground_info.query()

At this point we have: 
* Ability to search for site characteristics
* Campground location and site name

Nice to have:
* Drinking water availabilty
* Restroom access
* Current status - may be in facility description but not always

In [23]:
sc = Scraper("http://www.fs.usda.gov/recarea/mthood/recreation/camping-cabins/recarea/?recid=53228&actid=29", "Lost Lake")
sc.scrape()

{'FacilityStatus': 'Closed',
 'FacilityLatitude': '45.50080',
 'FacilityLongitude': '-121.81641',
 'FacilityElevation': '3200',
 'Conditions': 'CLOSED FOR THE SEASON\n\xa0\n**Lost Lake is currently limiting parking capacity within the campground and resort area to 50%.\xa0 Friday-Monday, a road barricade is staffed 4 miles below the entrance to the resort.\xa0 Once parking capacity has reached 50% vehicles will be turned around at that location.\xa0 This is the result of parking congestion along the roadways accessing the campground and resort.\xa0 The congestion has made it difficult for emergency personnel to access the site.**\xa0\xa0\nFor the 2020 season, site is operating at 50% capacity.\xa0 Capacity will be met early\xa0on\xa0weekends so strongly consider a mid-week visit instead.\xa0\nForest Road 13 is CLOSED at Lake Branch Bridge at mile post 10.8, just northwest of Lost Lake.',
 'Reservations': 'Reservations can be made by visiting Recreation.gov. \xa0Reservations must be mad

In [24]:
nf_sites = []
with open('../data/NF_sites/OR_sitelist.csv') as f:
    reader = DictReader(f)
    for row in reader:
        nf_sites.append(row)
nf_sites

[{'site_name': 'East Lemolo Campground',
  'site_url': 'https://www.fs.usda.gov/recarea/umpqua/recarea/?recid=63492'},
 {'site_name': 'Magone Lake Campground',
  'site_url': 'https://www.fs.usda.gov/recarea/malheur/recarea/?recid=39964'},
 {'site_name': 'East Davis Lake Campground',
  'site_url': 'https://www.fs.usda.gov/recarea/deschutes/recarea/?recid=38854'},
 {'site_name': 'Lost Lake Campground Resort and Day Use Area',
  'site_url': 'https://www.fs.usda.gov/recarea/mthood/recarea/?recid=53228'},
 {'site_name': 'Anthony Lake',
  'site_url': 'https://www.fs.usda.gov/recarea/wallowa-whitman/recarea/?recid=52199'},
 {'site_name': 'Musick Guard Station',
  'site_url': 'https://www.fs.usda.gov/recarea/umpqua/recarea/?recid=63428'},
 {'site_name': 'Lost Lake Campground',
  'site_url': 'https://www.fs.usda.gov/recarea/willamette/recarea/?recid=13362'}]

In [25]:
nf_data = []
for site in nf_sites:
    sc = Scraper(site['site_url'], site['site_name'])
    nf_data.append(sc.scrape())
nf_df = pd.DataFrame(nf_data)
nf_df

,FacilityStatus,FacilityLatitude,FacilityLongitude,FacilityElevation,Conditions,Reservations,FacilityName,Water,Restroom,Open Season
0,Temporarily Closed,43.310697,-122.162651,"4,150 feet",10/28/2020: Closed for the season. Will reopen in 2021,"Reservations can be made at www.recreation.gov for stays June through Labor Day. First come, fir...",East Lemolo Campground,NaN,NaN,NaN
1,Open,44.55266,-118.9094,5500,01/22/2021: The campground is is closed and the water has been turned off for the season. Pleas...,"To reserve the group site, visit www.recreation.gov or call 1-877-444-6777. Reservations can be...",Magone Lake Campground,Drinking Water,Vault Toilets,NaN
2,Closed,43.5867,-121.85667,4400,NaN,Reservations can be online through Recreation.gov or by calling toll free 1-877-444-6777 (Intern...,East Davis Lake Campground,Potable Water,Vault Toilet,NaN
3,Closed,45.50080,-121.81641,3200,CLOSED FOR THE SEASON\n \n**Lost Lake is currently limiting parking capacity within the campgrou...,Reservations can be made by visiting Recreation.gov. Reservations must be made 4 days ahead of ...,Lost Lake Campground Resort and Day Use Area,Drinking Water,Vault Toilet (18),NaN
4,Closed,44.9625128531073,-118.228574730768,7150,Current Conditions,https://anthonylakes.com/campgrounds/,Anthony Lake,Potable Water,Vault Toilets,July - September
5,Temporarily Closed,43.581026,-122.641745,"5,000 feet",10/09/2020- This site is currently closed per Forest Order 06-15-00-20-416,NaN,Musick Guard Station,NaN,NaN,Early Summer
6,Temporarily Closed,44.42927714677809,-121.912474623539,4200 feet,NaN,No advance reservations. All sites are first come first serve.,Lost Lake Campground,NaN,NaN,NaN


In [ ]:

campground_info.shape

In [26]:
# This should join left on campground_info
dm = distance_merge(nf_df, campground_info, 1500, 'ridb', 'nf')

/Users/gizmo/.pyenv/versions/3.8.5/envs/strata_venv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/Users/gizmo/.pyenv/versions/3.8.5/envs/strata_venv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [27]:
dm.FacilityName_nf.unique()

array([nan, 'Magone Lake Campground', 'East Davis Lake Campground',
       'Anthony Lake', 'Musick Guard Station'], dtype=object)

In [28]:
# putting it all together
ridb_facilities_url = "https://ridb.recreation.gov/api/v1/facilities"
params = {"activity_id":9, "state":"OR"}
headers = {"accept": "application/json", "apikey": "key"}


response = RequestsMock.get(ridb_facilities_url, params, headers=headers)
camping_json  = json.loads(response.text)

# Do we really need the campgrounds in a dataframe?
df_ridb_camping = pd.DataFrame(camping_json['RECDATA'])

campground_info = pd.DataFrame()
for facility in camping_json['RECDATA']:
    if facility.get('FacilityID') is not None:
        campground_url = f"{ridb_facilities_url}/{facility['FacilityID']}/campsites"
        resp = RequestsMock.get(campground_url, headers=headers)
        if resp.status_code != 200:
            continue
        
        campsites = json.loads(resp.text)
        if len(campsites['RECDATA']) > 0:
            df_campsites = pd.DataFrame(campsites['RECDATA'])
            campground_info = campground_info.append(df_campsites[['FacilityID', 'CampsiteID', 'CampsiteName', 'ATTRIBUTES']].merge(df_ridb_camping, on='FacilityID', how='left'))
            
nf_data = []
with open('../data/NF_sites/OR_sitelist.csv') as f:
    reader = DictReader(f)
    for row in reader:
        sc = Scraper(row['site_url'], row['site_name'])
        nf_data.append(sc.scrape())
nf_df = pd.DataFrame(nf_data)
merged = distance_merge(nf_df, campground_info, 2000, 'ridb', 'nf')

/Users/gizmo/.pyenv/versions/3.8.5/envs/strata_venv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [29]:
merged

,FacilityID,CampsiteID,CampsiteName,ATTRIBUTES,LegacyFacilityID,OrgFacilityID,ParentOrgID,ParentRecAreaID,FacilityName_ridb,FacilityDescription,...,FacilityStatus,FacilityLatitude_nf,FacilityLongitude_nf,FacilityElevation,Conditions,Reservations,FacilityName_nf,Water,Restroom,Open Season
0,251894,98358,008,"[{'AttributeName': 'Location Rating', 'AttributeValue': 'Good'}, {'AttributeName': 'Grills/Fire ...",135642,AN435642,131,1112,EAST LEMOLO CAMPGROUND,"<h2>Overview</h2>\nEast Lemolo is on the banks of Lemolo Lake, the highest reservoir on the Nort...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,251894,98441,014,"[{'AttributeName': 'Picnic Table', 'AttributeValue': 'Y'}, {'AttributeName': 'Checkout Time', 'A...",135642,AN435642,131,1112,EAST LEMOLO CAMPGROUND,"<h2>Overview</h2>\nEast Lemolo is on the banks of Lemolo Lake, the highest reservoir on the Nort...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,251894,98438,004,"[{'AttributeName': 'Picnic Table', 'AttributeValue': 'Y'}, {'AttributeName': 'Min Num of People'...",135642,AN435642,131,1112,EAST LEMOLO CAMPGROUND,"<h2>Overview</h2>\nEast Lemolo is on the banks of Lemolo Lake, the highest reservoir on the Nort...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,251894,98389,006,"[{'AttributeName': 'Picnic Table', 'AttributeValue': 'Y'}, {'AttributeName': 'Pets Allowed', 'At...",135642,AN435642,131,1112,EAST LEMOLO CAMPGROUND,"<h2>Overview</h2>\nEast Lemolo is on the banks of Lemolo Lake, the highest reservoir on the Nort...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,251894,98359,005,"[{'AttributeName': 'Placed on Map', 'AttributeValue': '1'}, {'AttributeName': 'Grills/Fire Ring'...",135642,AN435642,131,1112,EAST LEMOLO CAMPGROUND,"<h2>Overview</h2>\nEast Lemolo is on the banks of Lemolo Lake, the highest reservoir on the Nort...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,251434,96303,F001,"[{'AttributeName': 'Checkout Time', 'AttributeValue': '10:00 AM'}, {'AttributeName': 'Platform',...",125541,AN425541,131,1106,LOST LAKE RESORT AND CAMPGROUND,<h2>Overview</h2>\n<p>Lost Lake Campground is couched between beautiful Lost Lake and Mount Hood...,...,Closed,45.50080,-121.81641,3200,CLOSED FOR THE SEASON\n \n**Lost Lake is currently limiting parking capacity within the campgrou...,Reservations can be made by visiting Recreation.gov. Reservations must be made 4 days ahead of ...,Lost Lake Campground Resort and Day Use Area,Drinking Water,Vault Toilet (18),NaN
46,251434,96053,B011,"[{'AttributeName': 'Checkout Time', 'AttributeValue': '10:00 AM'}, {'AttributeName': 'Driveway L...",125541,AN425541,131,1106,LOST LAKE RESORT AND CAMPGROUND,<h2>Overview</h2>\n<p>Lost Lake Campground is couched between beautiful Lost Lake and Mount Hood...,...,Closed,45.50080,-121.81641,3200,CLOSED FOR THE SEASON\n \n**Lost Lake is currently limiting parking capacity within the campgrou...,Reservations can be made by visiting Recreation.gov. Reservations must be made 4 days ahead of ...,Lost Lake Campground Resort and Day Use Area,Drinking Water,Vault Toilet (18),NaN
47,251434,96013,B002,"[{'AttributeName': 'Driveway Length', 'AttributeValue': '99'}, {'AttributeName': 'Checkout Time'...",125541,AN425541,131,1106,LOST LAKE RESORT AND CAMPGROUND,<h2>Overview</h2>\n<p>Lost Lake Campground is couched between beautiful Lost Lake and Mount Hood...,...,Closed,45.50080,-121.81641,3200,CLOSED FOR THE SEASON\n \n**Lost Lake is currently limiting parking capacity within the campgrou...,Reservations can be made by visiting Recreation.gov. Reservations must be made 4 days ahead of ...,Lost Lake Campground Resort and Day Use Area,Drinking Water,Vault Toilet (18),NaN
48,251434,96009,D004,"[{'AttributeName': 'Grills/Fire Ring', 'AttributeValue': 'Y'}, {'AttributeName': 'Placed on Map'...",125541,AN425541,131,1106,LOST LAKE RESORT AND CAMPGROUND,<h2>Overview</h2>\n<p>Lost Lake Campground is couched between beautiful Lost 

In [ ]:
# add some queries